In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Realtime Color Detection Pipeline (OpenCV)
- Bước 1..6 đúng như pipeline bạn yêu cầu
- Thêm lựa chọn **mahalanobis** (2D trên a–b) ngoài **lab_polar** và **hsv_and_lab**
- Chỉ cấu hình trong class Config (không cần đổi trong UI camera). Không còn phím đổi mode/mask.
- Sửa lỗi CIEDE2000 TypeError (cài đặt thuần numpy)

Phím:
  q : quit
  a : toggle autoscale proc size
  p : lưu K-Means palette JSON
  s : snapshot
  m : toggle precise ↔ recall
  d : toggle ΔE method (ciede76 ↔ ciede2000)
  t : cycle COLOR_MASK_MODE (lab_polar ↔ hsv_and_lab ↔ mahalanobis)
snapshot
"""

import cv2
import json
import time
import math
from dataclasses import dataclass
from typing import Dict, Tuple, List
import numpy as np

# =========================
# 0) Config & Color Palette
# =========================

# 30-color palette (name -> hex)
BASE_COLORS: Dict[str, str] = {
    "red":"#ff0000","orange":"#ff7f00","yellow":"#ffff00","chartreuse":"#7fff00","green":"#00ff00",
    "springgreen":"#00ff7f","cyan":"#00ffff","azure":"#007fff","blue":"#0000ff","navy":"#000080",
    "indigo":"#4b0082","violet":"#8a2be2","purple":"#800080","magenta":"#ff00ff","rose":"#ff007f",
    "pink":"#ffc0cb","salmon":"#fa8072","crimson":"#dc143c","brown":"#8b4513","chocolate":"#d2691e",
    "gold":"#ffd700","olive":"#808000","lime":"#bfff00","teal":"#008080","turquoise":"#40e0d0",
    "sky":"#87ceeb","royalblue":"#4169e1","slateblue":"#6a5acd","beige":"#f5f5dc","black":"#000000"
}

@dataclass
class ModeThreshold:
    h_tol: int
    s_min: int
    v_min: int
    de_thresh: float

@dataclass
class LabPolarParams:
    de_thresh: float = 16.0
    hue_deg_tol: float = 12.0
    chroma_min_ratio: float = 0.7
    chroma_max_ratio: float = 1.3

@dataclass
class Config:
    # Target FPS & autoscale
    target_fps: float = 30.0
    autoscale: bool = True
    autoscale_margin: float = 2.0
    autoscale_interval: int = 30
    width_step: int = 32
    proc_width: int = 640
    min_width: int = 320
    max_width: int = 960

    # Preprocess
    use_grayworld: bool = True
    use_clahe: bool = True
    clahe_clip: float = 2.0
    clahe_grid: Tuple[int, int] = (8, 8)

    # Color labeling
    de_method: str = "ciede76"          # "ciede76" | "ciede2000"
    mode: str = "precise"                # "precise" | "recall" (chỉ ảnh hưởng preset tham số)
    COLOR_MASK_MODE: str = "lab_polar"   # "lab_polar" | "hsv_and_lab" | "mahalanobis"

    # Mahalanobis params (2D trên a-b)
    maha_k: float = 3.0                  # chấp nhận nếu d^2 <= k^2
    maha_sigma_base: float = 6.0         # sigma cơ bản cho a,b
    maha_sigma_scale_by_chroma: float = 0.06  # thêm theo C_ref
    maha_min_chroma: float = 6.0         # bỏ xám (C thấp)

    # Postprocess
    morph_open: int = 3
    morph_close: int = 5
    min_area_ratio: float = 0.0005

    # Palette
    k_palette: int = 6

# HSV + ΔE presets
MODE_PRECISE = ModeThreshold(h_tol=10, s_min=60, v_min=60, de_thresh=16.0)
MODE_RECALL  = ModeThreshold(h_tol=18, s_min=35, v_min=35, de_thresh=26.0)

# Lab-Polar presets
LP_PRECISE = LabPolarParams(de_thresh=16.0, hue_deg_tol=12.0, chroma_min_ratio=0.7, chroma_max_ratio=1.3)
LP_RECALL  = LabPolarParams(de_thresh=26.0, hue_deg_tol=20.0, chroma_min_ratio=0.5, chroma_max_ratio=1.6)


# =========================
# 1) Utilities
# =========================

def hex_to_bgr(hex_str: str) -> Tuple[int, int, int]:
    hex_str = hex_str.lstrip('#')
    r = int(hex_str[0:2], 16)
    g = int(hex_str[2:4], 16)
    b = int(hex_str[4:6], 16)
    return (b, g, r)


def bgr_to_lab_cv(bgr: Tuple[int, int, int]) -> np.ndarray:
    arr = np.uint8([[list(bgr)]])
    lab = cv2.cvtColor(arr, cv2.COLOR_BGR2LAB)
    return lab[0, 0].astype(np.float32)


def cv_lab_to_lab_float(lab_cv: np.ndarray) -> np.ndarray:
    lab_cv = lab_cv.astype(np.float32)
    L = lab_cv[..., 0] * (100.0 / 255.0)
    a = lab_cv[..., 1] - 128.0
    b = lab_cv[..., 2] - 128.0
    return np.stack([L, a, b], axis=-1)


def grayworld_white_balance(bgr: np.ndarray) -> np.ndarray:
    eps = 1e-6
    b, g, r = cv2.split(bgr)
    mb, mg, mr = b.mean() + eps, g.mean() + eps, r.mean() + eps
    m = (mb + mg + mr) / 3.0
    kb, kg, kr = m / mb, m / mg, m / mr
    b = cv2.multiply(b, kb)
    g = cv2.multiply(g, kg)
    r = cv2.multiply(r, kr)
    balanced = cv2.merge([b, g, r])
    return np.clip(balanced, 0, 255).astype(np.uint8)


def clahe_on_L(bgr: np.ndarray, clip: float = 2.0, grid: Tuple[int, int] = (8, 8)) -> np.ndarray:
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    L, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=grid)
    L2 = clahe.apply(L)
    lab2 = cv2.merge([L2, a, b])
    return cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

# ------- HSV utilities (giữ cho chế độ hsv_and_lab) -------

def compute_hsv_bounds(center_bgr: Tuple[int, int, int], h_tol: int, s_min: int, v_min: int) -> List[Tuple[np.ndarray, np.ndarray]]:
    arr = np.uint8([[list(center_bgr)]])
    hsv = cv2.cvtColor(arr, cv2.COLOR_BGR2HSV)[0, 0]
    h = int(hsv[0])
    lower_h = h - h_tol
    upper_h = h + h_tol
    ranges: List[Tuple[np.ndarray, np.ndarray]] = []
    if lower_h < 0:
        ranges.append((np.array([0, s_min, v_min]), np.array([upper_h % 180, 255, 255])))
        ranges.append((np.array([(180 + lower_h), s_min, v_min]), np.array([179, 255, 255])))
    elif upper_h > 179:
        ranges.append((np.array([0, s_min, v_min]), np.array([upper_h - 180, 255, 255])))
        ranges.append((np.array([lower_h, s_min, v_min]), np.array([179, 255, 255])))
    else:
        ranges.append((np.array([lower_h, s_min, v_min]), np.array([upper_h, 255, 255])))
    return ranges


def hsv_mask_for_color(hsv_img: np.ndarray, bgr_center: Tuple[int, int, int], mt: ModeThreshold) -> np.ndarray:
    ranges = compute_hsv_bounds(bgr_center, mt.h_tol, mt.s_min, mt.v_min)
    mask = np.zeros(hsv_img.shape[:2], dtype=np.uint8)
    for lo, hi in ranges:
        mask |= cv2.inRange(hsv_img, lo, hi)
    return mask

# ------- Lab ΔE -------

def deltaE76(lab_img_f: np.ndarray, lab_ref_f: np.ndarray) -> np.ndarray:
    diff = lab_img_f - lab_ref_f.reshape(1, 1, 3)
    return np.sqrt(np.sum(diff * diff, axis=2)).astype(np.float32)

# Vectorized CIEDE2000 (numpy-only)

def _ciede2000(lab_img_f: np.ndarray, lab_ref_f: np.ndarray) -> np.ndarray:
    L1 = lab_img_f[..., 0].astype(np.float32)
    a1 = lab_img_f[..., 1].astype(np.float32)
    b1 = lab_img_f[..., 2].astype(np.float32)

    L2 = np.float32(lab_ref_f[0])
    a2 = np.float32(lab_ref_f[1])
    b2 = np.float32(lab_ref_f[2])

    C1 = np.sqrt(a1 * a1 + b1 * b1)
    C2 = np.sqrt(a2 * a2 + b2 * b2)
    Cbar = (C1 + C2) / 2.0
    G = 0.5 * (1 - np.sqrt((Cbar ** 7) / (Cbar ** 7 + 25 ** 7)))
    a1p = (1 + G) * a1
    a2p = (1 + G) * a2
    C1p = np.sqrt(a1p * a1p + b1 * b1)
    C2p = np.sqrt(a2p * a2p + b2 * b2)

    h1p = (np.degrees(np.arctan2(b1, a1p)) + 360.0) % 360.0
    h2p = (np.degrees(np.arctan2(b2, a2p)) + 360.0) % 360.0

    dLp = L2 - L1
    dCp = C2p - C1p

    dhp = h2p - h1p
    dhp = np.where(dhp > 180, dhp - 360, dhp)
    dhp = np.where(dhp < -180, dhp + 360, dhp)
    dHp = 2.0 * np.sqrt(C1p * C2p) * np.sin(np.radians(dhp / 2.0))

    Lbar = (L1 + L2) / 2.0
    Cbarp = (C1p + C2p) / 2.0

    hsum = h1p + h2p
    hdiff = np.abs(h1p - h2p)
    hbarp = np.where((C1p * C2p) == 0, hsum, np.where(hdiff <= 180, hsum / 2.0, (hsum + 360.0) / 2.0))
    hbarp = hbarp % 360.0

    T = 1 - 0.17 * np.cos(np.radians(hbarp - 30)) + 0.24 * np.cos(np.radians(2 * hbarp)) \
        + 0.32 * np.cos(np.radians(3 * hbarp + 6)) - 0.20 * np.cos(np.radians(4 * hbarp - 63))

    SL = 1 + (0.015 * (Lbar - 50) ** 2) / np.sqrt(20 + (Lbar - 50) ** 2)
    SC = 1 + 0.045 * Cbarp
    SH = 1 + 0.015 * Cbarp * T

    delta_theta = 30.0 * np.exp(-((hbarp - 275.0) / 25.0) ** 2)
    RC = 2.0 * np.sqrt((Cbarp ** 7) / (Cbarp ** 7 + 25.0 ** 7))
    RT = -RC * np.sin(np.radians(2.0 * delta_theta))

    dE = np.sqrt((dLp / SL) ** 2 + (dCp / SC) ** 2 + (dHp / SH) ** 2 + RT * (dCp / SC) * (dHp / SH))
    return dE.astype(np.float32)


def deltaE(lab_img_f: np.ndarray, lab_ref_f: np.ndarray, method: str = "ciede76") -> np.ndarray:
    if method == "ciede2000":
        return _ciede2000(lab_img_f, lab_ref_f)
    return deltaE76(lab_img_f, lab_ref_f)

# ------- Lab-Polar mask -------

def angle_diff_deg(a, b):
    d = (a - b + 180.0) % 360.0 - 180.0
    return np.abs(d)


def lab_polar_from_lab_float(lab_f):
    a = lab_f[..., 1]
    b = lab_f[..., 2]
    chroma = np.sqrt(a * a + b * b)
    hue_deg = (np.degrees(np.arctan2(b, a)) + 360.0) % 360.0
    return hue_deg, chroma


def lab_polar_of_ref(lab_ref_f):
    a = float(lab_ref_f[1]); b = float(lab_ref_f[2])
    chroma = math.sqrt(a * a + b * b)
    hue_deg = (math.degrees(math.atan2(b, a)) + 360.0) % 360.0
    return hue_deg, chroma


def lab_polar_mask(lab_f, lab_ref_f, params: LabPolarParams, de_method="ciede76"):
    dE = deltaE(lab_f, lab_ref_f, method=de_method)
    hue_img, chroma_img = lab_polar_from_lab_float(lab_f)
    hue_ref, chroma_ref = lab_polar_of_ref(lab_ref_f)

    delta_h = angle_diff_deg(hue_img, hue_ref)
    cmin = params.chroma_min_ratio * chroma_ref
    cmax = params.chroma_max_ratio * chroma_ref

    m = (dE <= params.de_thresh) & (delta_h <= params.hue_deg_tol) & (chroma_img >= cmin) & (chroma_img <= cmax)
    return m.astype(np.uint8) * 255

# ------- Mahalanobis (a,b) + ΔE guardrail -------

def mahalanobis_mask(lab_f: np.ndarray, lab_ref_f: np.ndarray, cfg: Config) -> np.ndarray:
    a_img = lab_f[..., 1]
    b_img = lab_f[..., 2]
    a0 = float(lab_ref_f[1])
    b0 = float(lab_ref_f[2])
    C_ref = math.sqrt(a0*a0 + b0*b0)

    sigma = cfg.maha_sigma_base + cfg.maha_sigma_scale_by_chroma * C_ref
    sigma2 = sigma * sigma
    k2 = cfg.maha_k * cfg.maha_k

    d2 = ((a_img - a0)**2 + (b_img - b0)**2) / sigma2
    m_maha = (d2 <= k2)

    preset = MODE_PRECISE if cfg.mode == "precise" else MODE_RECALL
    dE = deltaE(lab_f, lab_ref_f, method=cfg.de_method)
    chroma_img = np.sqrt(a_img*a_img + b_img*b_img)

    m = m_maha & (dE <= preset.de_thresh) & (chroma_img >= cfg.maha_min_chroma)
    return m.astype(np.uint8) * 255

# ------- Postprocess & Palette -------

def morph_refine(mask: np.ndarray, open_ks: int, close_ks: int) -> np.ndarray:
    m = mask
    if open_ks > 1:
        k1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_ks, open_ks))
        m = cv2.morphologyEx(m, cv2.MORPH_OPEN, k1)
    if close_ks > 1:
        k2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_ks, close_ks))
        m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, k2)
    return m


def remove_small_blobs(mask: np.ndarray, min_area: int) -> np.ndarray:
    if min_area <= 1:
        return mask
    num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    keep = np.zeros(num, dtype=np.bool_)
    keep[0] = False
    for i in range(1, num):
        keep[i] = stats[i, cv2.CC_STAT_AREA] >= min_area
    return (keep[labels]).astype(np.uint8) * 255


def kmeans_palette_lab(lab_img_cv: np.ndarray, k: int = 6, attempts: int = 3) -> np.ndarray:
    data = lab_img_cv.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    _, labels, centers = cv2.kmeans(data, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
    return centers.astype(np.uint8)


def lab_cv_to_hex(lab_cv: np.ndarray) -> str:
    lab_cv = lab_cv.astype(np.uint8).reshape((1, 1, 3))
    bgr = cv2.cvtColor(lab_cv, cv2.COLOR_LAB2BGR)[0, 0]
    r, g, b = int(bgr[2]), int(bgr[1]), int(bgr[0])
    return f"#{r:02x}{g:02x}{b:02x}"


# =========================
# 2) Main Loop
# =========================

def main():
    cfg = Config()

    # Prepare color references
    color_refs: Dict[str, Dict[str, np.ndarray]] = {}
    for name, hx in BASE_COLORS.items():
        bgr = hex_to_bgr(hx)
        lab_cv = bgr_to_lab_cv(bgr)
        lab_f = cv_lab_to_lab_float(lab_cv)
        color_refs[name] = {
            "bgr": np.array(bgr, dtype=np.uint8),
            "lab_cv": lab_cv.astype(np.uint8),
            "lab_f": lab_f.astype(np.float32),
        }

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("[Error] Cannot open camera.")
        return

    ema_fps = None
    last_t = time.time()
    frame_count = 0

    win_name = "Realtime Color Detection"
    cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

    while True:
        ok, frame = cap.read()
        if not ok:
            print("[Warn] Empty frame.")
            break

        H0, W0 = frame.shape[:2]
        scale = cfg.proc_width / float(W0)
        proc_h = max(1, int(round(H0 * scale)))
        proc_w = cfg.proc_width
        frame_proc = cv2.resize(frame, (proc_w, proc_h), interpolation=cv2.INTER_AREA)

        # 2) Preprocess
        if cfg.use_grayworld:
            frame_proc = grayworld_white_balance(frame_proc)
        if cfg.use_clahe:
            frame_proc = clahe_on_L(frame_proc, clip=cfg.clahe_clip, grid=cfg.clahe_grid)

        # Convert once
        hsv = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2HSV) if cfg.COLOR_MASK_MODE == "hsv_and_lab" else None
        lab_cv = cv2.cvtColor(frame_proc, cv2.COLOR_BGR2LAB)
        lab_f = cv_lab_to_lab_float(lab_cv)

        # thresholds & areas
        proc_area = proc_w * proc_h
        min_area = int(cfg.min_area_ratio * proc_area)

        detections: List[Tuple[str, Tuple[int, int, int, int], int]] = []

        for name, ref in color_refs.items():
            if cfg.COLOR_MASK_MODE == "lab_polar":
                lpp = LP_PRECISE if cfg.mode == "precise" else LP_RECALL
                mask = lab_polar_mask(lab_f, ref["lab_f"], lpp, de_method=cfg.de_method)
            elif cfg.COLOR_MASK_MODE == "hsv_and_lab":
                mt = MODE_PRECISE if cfg.mode == "precise" else MODE_RECALL
                mask_hsv = hsv_mask_for_color(hsv, tuple(int(x) for x in ref["bgr"]), mt)
                dE = deltaE(lab_f, ref["lab_f"], cfg.de_method)
                mask_de = (dE <= mt.de_thresh).astype(np.uint8) * 255
                mask = cv2.bitwise_and(mask_hsv, mask_de)
            elif cfg.COLOR_MASK_MODE == "mahalanobis":
                mask = mahalanobis_mask(lab_f, ref["lab_f"], cfg)
            else:
                mask = lab_polar_mask(lab_f, ref["lab_f"], LP_PRECISE if cfg.mode=="precise" else LP_RECALL, de_method=cfg.de_method)

            # 4) Postprocess
            mask = morph_refine(mask, cfg.morph_open, cfg.morph_close)
            mask = remove_small_blobs(mask, min_area)

            num, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
            for i in range(1, num):
                x, y, w, h, area_px = stats[i]
                if area_px < min_area:
                    continue
                detections.append((name, (x, y, w, h), int(area_px)))

        # Draw
        disp = frame.copy()
        sx = W0 / float(proc_w)
        sy = H0 / float(proc_h)

        for name, (x, y, w, h), area_px in detections:
            X = int(round(x * sx)); Y = int(round(y * sy))
            W = int(round(w * sx)); H = int(round(h * sy))
            color_bgr = tuple(int(c) for c in color_refs[name]["bgr"])  # BGR
            cv2.rectangle(disp, (X, Y), (X + W, Y + H), color_bgr, 2)
            area_ratio = (area_px / float(proc_area)) * 100.0
            label = f"{name} {area_ratio:.1f}%"
            tsize, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(disp, (X, Y - tsize[1] - 6), (X + tsize[0] + 6, Y), color_bgr, -1)
            cv2.putText(disp, label, (X + 3, Y - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

        # HUD
        now = time.time()
        dt = max(1e-6, now - last_t)
        inst_fps = 1.0 / dt
        last_t = now
        ema_fps = inst_fps if ema_fps is None else 0.9 * ema_fps + 0.1 * inst_fps
        frame_count += 1

        if cfg.autoscale and frame_count % cfg.autoscale_interval == 0:
            if ema_fps < (cfg.target_fps - cfg.autoscale_margin) and cfg.proc_width > cfg.min_width:
                cfg.proc_width = max(cfg.min_width, cfg.proc_width - cfg.width_step)
            elif ema_fps > (cfg.target_fps + cfg.autoscale_margin) and cfg.proc_width < cfg.max_width:
                cfg.proc_width = min(cfg.max_width, cfg.proc_width + cfg.width_step)

        hud1 = f"FPS: {ema_fps:.1f} | target: {cfg.target_fps:.0f} | mode: {cfg.mode} | mask: {cfg.COLOR_MASK_MODE} | ΔE: {cfg.de_method}"
        hud2 = f"proc: {proc_w}x{proc_h} (auto:{'on' if cfg.autoscale else 'off'}) | open={cfg.morph_open} close={cfg.morph_close} | min_area={min_area}"
        cv2.putText(disp, hud1, (10, 24), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(disp, hud1, (10, 24), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(disp, hud2, (10, 48), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(disp, hud2, (10, 48), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1, cv2.LINE_AA)

        cv2.imshow(win_name, disp)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            cfg.autoscale = not cfg.autoscale
            print(f"[Autoscale] {'ON' if cfg.autoscale else 'OFF'}")
        elif key == ord('s'):
            ts = int(time.time())
            out_path = f"snapshot_{ts}.png"
            cv2.imwrite(out_path, disp)
            print(f"[Saved] {out_path}")
        elif key == ord('p'):
            centers = kmeans_palette_lab(lab_cv, k=cfg.k_palette, attempts=5)
            items = []
            for i in range(centers.shape[0]):
                lab_u8 = centers[i]
                hex_col = lab_cv_to_hex(lab_u8)
                lab_f_center = cv_lab_to_lab_float(lab_u8.reshape(1,1,3)).reshape(3).tolist()
                items.append({"lab_cv": centers[i].astype(int).tolist(), "lab": lab_f_center, "hex": hex_col})
            ts = int(time.time())
            js = {"timestamp": ts, "k": cfg.k_palette, "items": items}
            out_json = f"palette_{ts}.json"
            with open(out_json, "w", encoding="utf-8") as f:
                json.dump(js, f, ensure_ascii=False, indent=2)
            print(f"[Palette] Saved {out_json}")
        elif key == ord('m'):
            cfg.mode = 'recall' if cfg.mode == 'precise' else 'precise'
            print(f"[Mode] {cfg.mode}")
        elif key == ord('d'):
            cfg.de_method = 'ciede2000' if cfg.de_method == 'ciede76' else 'ciede76'
            print(f"[ΔE] {cfg.de_method}")
        elif key == ord('t'):
            order = ['lab_polar', 'hsv_and_lab', 'mahalanobis']
            try:
                idx = order.index(cfg.COLOR_MASK_MODE)
            except ValueError:
                idx = 0
            cfg.COLOR_MASK_MODE = order[(idx + 1) % len(order)]
            print(f"[Mask] {cfg.COLOR_MASK_MODE}")

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


2025-08-25 10:53:41.920 Python[10033:656046] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


[Mode] recall
[Mode] precise
[Mask] hsv_and_lab
[Mode] recall
[Mask] mahalanobis
[Mode] precise
